In [103]:
import numpy as np
from matplotlib import pyplot as plt, animation

from nonlinear.gauss_seidel import GaussSeidel
from nonlinear.jacobi import Jacobi
from nonlinear.solve_nonlinear import FSolve
from ode.euler_schemes import ImplicitEuler
from ode.solve_ivp import solve_ivp_implicit


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
plt.rcParams.update({
    "text.usetex": True,
    "animation.html": "jshtml"
})

## Problem definition

In [105]:
g = 9.81
m1 = 1
m2 = 1
l1 = 1
l2 = 1

In [106]:
def f(_: float, y: np.ndarray) -> np.array:
    y1,y2,y3,y4 = y
    return np.array([
        y3,
        y4,
        (-g*(2*m1 + m2)*np.sin(y1) - m2*g*np.sin(y1 - 2*y2) - 2*np.sin(y1 - y2)*m2*(y4*y4*l2 + y3*y3*l1*np.cos(y1 - y2)))
            / (l1*(2*m1 + m2 - m2*np.cos(2*y1 - 2*y2))),
        (2*np.sin(y1 - y2)*(y3*y3*l1*(m1 + m2) + g*(m1 + m2)*np.cos(y1) + y4*y4*l2*m2*np.cos(y1 - y2)))
            / (l2*(2*m1 + m2 - m2*np.cos(2*y1 - 2*y2)))
    ])

## Simulation parameters

In [107]:
kmax = 3                            # Amount of iterations
n = 200                             # Time steps
T = 4*np.pi*np.sqrt((l1+l2)/g)      # Time interval
t = np.linspace(0, T, n)
scheme = 'gauss_seidel'

## Solution
### Solve dynamic iteration

In [108]:
if scheme == 'jacobi':
    solver = Jacobi(kmax)
elif scheme == 'gauss_seidel':
    solver = GaussSeidel(kmax)
elif scheme == 'fsolve':
    solver = FSolve()
else:
    raise ValueError(f"Value of option <scheme> was {scheme} but is not allowed!")

In [109]:
y0 = np.array([np.pi/6,0,0,0])
res = solve_ivp_implicit(t, y0, ImplicitEuler(f), solver)

In [110]:
theta1 = res[:,0]
theta2 = res[:,1]

x1 = l1*np.sin(theta1)
y1 = -l1*np.cos(theta1)

x2 = l2*np.sin(theta2) + x1
y2 = -l2*np.cos(theta2) + y1

### Animate pendulum

In [111]:
%%capture

l = l1 + l2
fig = plt.figure(figsize=(5,4))
ax = fig.add_subplot(autoscale_on=False, xlim=(-l, l), ylim=(-1.5*l, 0.5*l))
ax.set_aspect('equal')
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
trace, = ax.plot([], [], '.-', lw=1, ms=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

In [112]:
def animate(i):
    xi = [0, x1[i], x2[i]]
    yi = [0, y1[i], y2[i]]

    history_x = x2[:i]
    history_y = y2[:i]

    line.set_data(xi, yi)
    trace.set_data(history_x, history_y)
    time_text.set_text(time_template % t[i])
    return line, trace, time_text

In [ ]:
animation.FuncAnimation(fig, animate, frames=theta1.size, interval=30)